In [1]:
import nltk.tokenize as tk
def word_tokenize(text):
    words = tk.word_tokenize(text)
    return len(words), words

In [2]:
# Call on concatenation of body and title
from collections import namedtuple
tokenized_row = namedtuple('tokenized_row', 'sent_count sentences word_count words')

In [3]:
from gensim.models.doc2vec import Doc2Vec
import multiprocessing
from collections import namedtuple

Document = namedtuple('ScoreDocument', 'words tags')

filenames = ['combined_train_test.p', 'r_train_so_test.p', 'so_train_r_test.p',
            'so_alone.p', 'reddit_alone.p']
cores = multiprocessing.cpu_count()

c:\program files\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [23]:
import pickle
def import_dict(token_dict, titles, bodies, title_sentences, body_sentences):
    for index in token_dict:
        title, body = token_dict[index]
        titles.append(Document(title.words, tags=[index]))
        bodies.append(Document(body.words, tags=[index]))
        count = 0
        for sentence in title.sentences:
            title_sentences.append(Document(word_tokenize(sentence)[1], tags=[str(index) + '_' +str(count)]))
            count += 1
        count = 0
        for sentence in body.sentences:
            body_sentences.append(Document(word_tokenize(sentence)[1], tags=[str(index) + '_' +str(count)]))
            count += 1
    return titles, bodies, title_sentences, body_sentences

for filename in filenames:
    directory_name = filename.split('.p')[0]
    with open(directory_name + "/tokenized_dict.p", 'rb') as pfile:
        train_token_dict, test_token_dict = pickle.load(pfile)
    titles = []
    title_sentences = []
    bodies = []
    body_sentences = []
    titles, bodies, title_sentences, body_sentences = import_dict(train_token_dict, titles, bodies, title_sentences, body_sentences)
    titles, bodies, title_sentences, body_sentences = import_dict(test_token_dict, titles, bodies, title_sentences, body_sentences) 

    model = Doc2Vec(titles, size=100, workers=cores, window=8, min_count=5)
    model.save(directory_name + '/titles.doc2vec')
    model = Doc2Vec(bodies, size=100, workers=cores, window=8, min_count=5)
    model.save(directory_name + '/bodies.doc2vec')
    model = Doc2Vec(body_sentences, size=100, workers=cores, window=8, min_count=5)
    model.save(directory_name + '/body_sentences.doc2vec')
    model = Doc2Vec(title_sentences, size=100, workers=cores, window=8, min_count=5)
    model.save(directory_name + '/title_sentences.doc2vec')

c:\program files\python36\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
